In [5]:
import geopandas as gpd
import pandas as pd
import os
import io
import requests
import boto3

This script intitate the dataframes that will store the different indicators calauclated for urbanshift cities.
- Input: georef file containing the list of cities' names and that we use for retrieving the boundaries files from the aws s3 (you can find more information about the georeferential-file in [this wiki page](https://github.com/wri/cities-urbanshift/wiki/Tutorial#administrative-boundaries))
- Output: A data frame containing the list of cities and sub-cities with their uniques identifies. This dataframe will be used as a based for integrating the indicators. It is stored in aws s3 bucket.

Run the script only when you want to initialize all the indicators.

# Load input data

In [6]:
# get list of urbanshift cities
boundary_georef = pd.read_csv('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
boundary_georef = boundary_georef.dropna(subset=['units_boundary_name']).reset_index(drop=True)
boundary_georef.head()

,geo_name,level,aoi_boundary_name,units_boundary_name,city_name,country_name,country_code,continent
0,ARG-Mendoza,region,ADM3union,ADM3,Mendoza,Argentina,ARG,America
1,ARG-Mar_del_Plata,city,ADM3,ADM4,Mar del Plata city,Argentina,ARG,America
2,ARG-Ushuaia,city,ADM4,ADM5,Ushuaia city,Argentina,ARG,America
3,ARG-Salta,region,ADM2union,ADM3,Salta,Argentina,ARG,America
4,ARG-Buenos_Aires,region,ADM2union,ADM2,Buenos Aires,Argentina,ARG,America


# Fill initial indicator table with the list of features

In [7]:
# create empty df
cities_indicators = pd.DataFrame()

In [8]:
# populate the df with the list of geo identifiers
for i in range(0, len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    # get boundary area of interest
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_path = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_data = gpd.read_file(boundary_path)
    boundary_data = boundary_data[["geo_id","geo_level", "geo_name","geo_parent_name"]]
    
    # append to dataframe
    cities_indicators = pd.concat([cities_indicators, boundary_data])
    
    # get boundary unit of analysis
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
    boundary_path = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_data = gpd.read_file(boundary_path)
    boundary_data = boundary_data[["geo_id","geo_level", "geo_name","geo_parent_name"]]
    
    # append to dataframe
    cities_indicators = pd.concat([cities_indicators, boundary_data])
    

0

 geo_name: ARG-Mendoza
1

 geo_name: ARG-Mar_del_Plata
2

 geo_name: ARG-Ushuaia
3

 geo_name: ARG-Salta
4

 geo_name: ARG-Buenos_Aires
5

 geo_name: BRA-Teresina
6

 geo_name: BRA-Teresina
7

 geo_name: BRA-Florianopolis
8

 geo_name: BRA-Florianopolis
9

 geo_name: BRA-Belem
10

 geo_name: BRA-Belem
11

 geo_name: CRI-San_Jose
12

 geo_name: RWA-Kigali
13

 geo_name: SLE-Freetown
14

 geo_name: SLE-Freetown
15

 geo_name: MAR-Marrakech
16

 geo_name: IND-Chennai
17

 geo_name: IND-Chennai
18

 geo_name: IND-Pune
19

 geo_name: IND-Surat
20

 geo_name: CHN-Chengdu
21

 geo_name: CHN-Chongqing
22

 geo_name: CHN-Ningbo
23

 geo_name: IDN-Jakarta
24

 geo_name: IDN-Bitung
25

 geo_name: IDN-Semarang
26

 geo_name: IDN-Balikpapan
27

 geo_name: IDN-Palembang


In [9]:
cities_indicators.head()

,geo_id,geo_level,geo_name,geo_parent_name
0,ARG-Mendoza_ADM-3-union_1,ADM-3-union,ARG-Mendoza,ARG-Mendoza
0,ARG-Mendoza_ADM-3_1,ADM-3,Distrito Las Barrancas,ARG-Mendoza
1,ARG-Mendoza_ADM-3_2,ADM-3,Distrito San Roque,ARG-Mendoza
2,ARG-Mendoza_ADM-3_3,ADM-3,Distrito Fray Luis Beltrán,ARG-Mendoza
3,ARG-Mendoza_ADM-3_4,ADM-3,Distrito Rodeo del Medio,ARG-Mendoza


# Upload in aws s3

In [10]:
# connect to s3
aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [13]:
# upload to aws
key_data = 'indicators/cities_indicators.csv'
bucket_name = 'cities-urbanshift' 
cities_indicators.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [14]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')